efore you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the related LOs requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>CS152 Assignment 3: The DPLL Algorithm</h1>

<h1>Question 1</h1>

Define your <code>Literal</code> class below.  Ensure that you define a <code>name</code> and <code>sign</code> attribute as required in the assignment description.  In addition, include any other attributes and member functions as needed. You will need to overload <code>__neg__()</code> to correctly handle negated literals.  You may need to overload <code>__eq__()</code> and <code>__hash__()</code> also.

In [1]:
# Import any packages you need here

#import copy to update KB without changing old version
import copy

# Also define any variables as needed




class Literal:
    """
    Literal class: defines attributes name and sign for a literal (default of sign is True)
    defines negation, equality, hash, representation and greater than to serve different purposes in the DPLL algorithm
    """
    
    def __init__(self, name, sign = True):
        #defines the name and the sign of the literal 
        #teh default of the sign is True
        self.name = name
        self.sign = sign
        
    def __neg__(self):
        #defines a negative of a literal as a literal with the same name and sign as negative
        #represents the negation of a literal
        return Literal(self.name, sign = False)
        
    def __eq__(self, other):
        #defines that two literals are equal if there name is equal 
        #negated litetals are equal to positive literals
        return self.name == other.name
    
    def __hash__(self):
        #defines that hashing happens based on the name
        return hash(self.name)
    
    def __repr__(self):
        #defines the representation of the literal as its name
        return self.name
        
    def __gt__(self, other):
        #defins that we compare two literals alphabetically based on their name
        return self.name > other.name

<h1>Question 2</h1>
Implement DPLL by filling in the stubs below.  Note that the various heuristics are not required to be implemented for basic functionality, but a template has been provided for you to do so if you attempt the extension questions

In [2]:
# Define degree, pure symbol and unit clause heuristics here (optional).
# Add in your functions under the templates below

#Heuristic Functions 

def degree(KB):
    """
    degree heuristic: This function takes a KB as input and returns a literal based on the degree heuristic
    - returns literal that occurs in the most number of clauses in the KB
    - by equal occurrence it chooses the alphabetically first literal
    """
    
    #list to store all literals
    literal_list = []
    #current max of occurences of one literal
    max_count = 0
    #literal with the current max count
    max_literal = None
    
    #loop over each sentence and each literal within them
    for sentence in KB:
        for literal in sentence:
            #store literals in a list
            literal_list.append(literal)
    
    #loop over all unique literals in KB
    for literal in set.union(*KB):
        #if current literal is in more sentences than current max: do updates
        if literal_list.count(literal) > max_count:
            #update max count and literal to values of current literal
            max_count = literal_list.count(literal)
            max_literal = literal
        #if current literal is in the same amout of sentences than current max and comes alphabetically first: do updates
        elif literal_list.count(literal) == max_count and literal < max_literal:
            #update max count and literal to values of current literal
            max_count = literal_list.count(literal)
            max_literal = literal
    
    #return the max literal
    return max_literal


def pure_symbol(KB, heuristic):
    """
     pure symbol heuristic: takes KB and heuristic as input and returns a pure symbol 
    - a pure symbol is a symbol that always occurs with the same sign in the KB
    - if more than one pure symbol is in KB, the heuristic parameter defines how to choose 
        - if heuristic is 2: choose alphabetically first literal
        - if heuristic is 3: choose literal with the most occurrences in KB
    - if there is no pure symbol, return a unit clause according to the unit class function
    """
    
    #list to store all literals
    literal_list = []
    #list to store all pure symbols
    pure_symbols = []
    
    #loop over each sentence and literal within them
    for sentence in KB:
        for literal in sentence:
            #add all literals to literal list
            literal_list.append(literal)
    
    #set max count of occurences of pure symbol to 0
    max_count = 0
    #loop over each unique literal in KB
    for literal in set.union(*KB):
        #set the sign of the literal to None
        sign = None
        #set pure to True
        pure = True
        #start count at 0
        count = 0
        #loop over each literal in literal list
        for i in literal_list:
            #check if current unique literal matches literal in list
            if literal == i:
                #if sign is not defined yet set it to the sign of the literal i and increase count
                if sign == None:
                    sign = i.sign
                    count +=1
                #otherwise check if sign of i and literal are the same
                else:
                    #if sign are not the same set pure to False and increaes count otherwise
                    if sign != i.sign:
                        pure = False
                    else:
                        count +=1
        
        #check if literal is pure add it to pure symbols list
        if pure:
            pure_symbols.append(literal)
            
        #if we use heuristic 3 check for max
        if heuristic == 3:
            # if the count is higher than the current count or equal and the current literal comes alphabetically first
            if count > max_count or (count == max_count and literal < max_literal):
                #update max count and literal 
                max_count = count
                max_literal = literal
                
    #if there is one pure symbol return it
    if len(pure_symbols) == 1:
        return pure_symbols[0]
    
    #if there is no pure symbol return the result of the unitclause function (input KB)
    elif len(pure_symbols) == 0:
        return unitclause(KB)
     
    else:
        #check if heuristic 3 is used and return the max literal if it is
        if heuristic == 3:
            return max_literal
        #return the alphabetically first pure symbol otherwise
        else:
            return sorted(pure_symbols)[0]
        
    
def unitclause(KB):
    """
    unit clause heuristic: takes a KB as input and returns a unit clause 
    - chooses the alphabetiaclly first unit clause if more than one exists 
    - a unit clause is a clause with just one unassigned literal 
    """
    
    #list for all unit clauses
    unit_clauses = []
    
    #loop over sentences in KB
    for sentence in KB:
        #if sentences have one literal add them to unit clauses list
        if len(sentence) == 1:
            unit_clauses.append(next(iter(sentence)))
    
    #if there is just one unit clause return it
    if len(unit_clauses) == 1:
        return unit_clauses[0]
    #if there is more than one unit clause return alphabetically first one
    elif len(unit_clauses) > 1:
        return sorted(unit_clauses)[0]
    #if there are no unit clauses return the degree heuristic with KB as input
    else:
        return degree(KB)

def update(KB, new_literal, truth_value):
    """
    update function: takes a KB, a literal, and a truth value as input and returns the updated KB 
    - assumes that we assign the new_literal to the truth_value
    - if a sentence has the literal with the same truth value, the sentence is true and we remove it from the KB
    - if a sentence has the literal with different truth value we remove the literal from the sentence 
    - if a sentence is emptty but not removed (so true), the kb is not satisfyable and we retrun False
    """
    
    #list of sentences to remove
    sentences_remove = []
    #list of sentences to update
    sentences_update = []
    #define new KB
    new_KB = []
    
    #loop over all senteces in KB
    for sentence in range(len(KB)):
        #defins if sentence has the newly assigned literal (initially false)
        has_lit = False
        #loop over all literlas in sentence
        for literal in KB[sentence]:
            #if literal is the new literal, we store that the sentence has the literal
            if literal == new_literal:
                has_lit = True
                #if the truth values of literal is different than the assigned one, we update the sentence
                if truth_value != literal.sign:
                    #copy sentence and remove literl in copy
                    updated_sen = KB[sentence].copy()
                    updated_sen.remove(literal)
                    #if sentence is empty KB is unsatisfyable and we return False
                    if len(updated_sen) == 0:
                        return False
                    #we append updated sentence to the new KB
                    new_KB.append(updated_sen)
        
        #if a sentence does not have the new literal, we append the sentence to the new KB
        if not has_lit:
            new_KB.append(KB[sentence])
    
    #return the new KB
    return new_KB    
    

def get_symbol_list(sl):
    """
    get sat model function: takes the symbol list as defined by my DPLL as input and returns it as defined in tests
    - my implementation stores one symbol for each assignment of a literal to true and false (other symbols unchanged) 
            - example: if we define A first to True and than to False we get [A,A]
    - the test implementation stores one symbol once for true and false if no otehr symbols are unchanged 
            - example: if we define A first to True and than to False we get [A]
    - removed all double entries resulting from same branch in search tree
    """
    #return symbol list if it is empty 
    if len(sl) == 0:
        return sl
    
    #define new symbol list and add first literal from original symbol list
    new_sl = [sl[0]]
    #define the difference of the positions we have to compare to while we backtracked 
    diff = 1
    #loop over all put the first literal in the symbol list
    for i in range(1, len(sl)):
        #if literal is equal to the literal in the difference position in the new list increase difference
        if sl[i] == new_sl[-diff]:
            if diff < len(new_sl):
                diff +=1
        #if symbol is not equal to symbol position append it to new symbol list and set difference back to 1
        else:
            new_sl.append(sl[i])
            diff = 1
    
    #return new symbol list
    return new_sl
    

def helper(KB, truth_value, sat_model, symbol_list, heuristic_level):
    """
    helper function for the recursion:
    input: a KB, a truth value, a symbol list, and a heurstic level
    output: False if KB is not satisfyable
            (True, sat_model, symbol_list) if satisfyable
            
    - recursively searches in a depth first manner for sat model that satisfies KB
    - first literal assignment cannot be changed (result is result for one half of complete search tree)
    - updates symbol list and sat model during recursions 
    - terminates once a satisfiable model is found or when no satisfiable model is possible
    - always assigns newly picked literal first to true than to false

    """
    #if the KB is False, we return False
    if KB == False:
        return False
    
    #pick new literal according to heuristic 
    #if heuristic is 0: new listeral is first literal found in KB
    if heuristic_level == 0:
        new_literal = next(iter(KB[0]))
    #if heuristic is 1 use dgree function
    elif heuristic_level == 1:
        new_literal = degree(KB)
    #if heuristic level is 2 or 3 use pure symbol function and input heursitic level
    elif heuristic_level == 2 or heuristic_level == 3:
        new_literal = pure_symbol(KB, heuristic_level)
   
    #update sat model by assigning new literal to truth value
    sat_model[new_literal] = truth_value
    #update KB using update_kb function and passing teh KB, the literal chose and the truth value defined
    updated_kb = update(KB, new_literal, truth_value)
    #append new literal to symbokl list
    symbol_list.append(new_literal)
    
    #if updated kb is not False and empty we found model that satisfies KB
    if updated_kb != False and len(updated_kb) == 0:
        #return True, model, and symbol list
        return (True, sat_model, symbol_list)
    
    #define true path by calling helper function recursively, with updated Kb and truth value true
    true_path = helper(updated_kb, True, sat_model, symbol_list, heuristic_level)
    #if true path is not false we return it
    if true_path != False:
        return true_path
    #if true path is false we go to false_path with updated KB and truth value false (again, recursive call to helper)
    else:
        false_path =  helper(updated_kb, False, sat_model, symbol_list, heuristic_level)
        #if fale path is not flase we return it
        if false_path != False:
            return false_path
        #if both paths are flase we return false
        else:
            return False

        
def DPLL_Satisfiable(KB, heuristic_level=0):
    ''' satisfiable, model = DPLLSatisfiable(KB)
        Takes in a KB and returns whether the KB is satisfiable, and the model that makes it so
        
        KB: A knowledge base of clauses (CNF) consisting of a list of sets of literals.  A KB might look like
            [{A,B},{-A,C,D}]
        heuristic_level: An integer that will be passed in to specify which heuristics to implement 
            (only for the extension activities).
        satisfiable: Returns True if the KB is satisfiable, or False otherwise
        Model: A dictionary that assigns a truth value to each literal for the model that satisfies KB.
            For example, a model might look like {A: True, B: False}
    '''
    #define dict for sat model
    sat_model = {}
    #define list for symbol list
    symbol_list = []
    # define truth path by calling helper function (setting first chosen literl to true)
    tp =  helper(KB,True, sat_model, symbol_list, heuristic_level)
    #if truth path is satisfyable, we return (True, sat model, symbol list)
    if tp != False:
        return (tp[0], tp[1], get_symbol_list(tp[2]))
    #if truth path is not satisfyable we find false path (setting first literal to false)
    else:
        fp = helper(KB,False, sat_model, symbol_list, heuristic_level)
        #if false path is satisfyable, we return (True, sat model, symbol list)
        if fp != False:
            return (fp[0], fp[1], get_symbol_list(fp[2]))
        #if both paths are not satisfyable, we return (False, sat model(empty), symbol list(empty))
        else:
            return (False, {}, [])
    
    


<h1>Question 3</h1>

Implement your KB from Russell & Norvig in CNF by filling in the sets inside the list <code>KB</code> below.  Ensure that your KB is in a list of set format as stated in the assignment instructions.

In [3]:
# Define some literals
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')

# YOUR CODE HERE
#define KB with sentences in CNF
KB = [{-A, B, E}, {A,-B}, {A, -E}, {-E, D}, {-C, -F, -B}, {-E, B}, {-B, F}, {-B, C}]

#tets result of KB using DPLL
print(DPLL_Satisfiable(KB, heuristic_level = 0))


(True, {B: False, A: False, D: False, F: False, C: False, E: False}, [B, A, D, F, C, E, F, C, A, B, A, E, E])


<h1>Extensions</h1>

1. Implement the degree heuristic for choosing symbols.  If <code>heuristic_level=1</code>, then the degree heuristic should be used to select which symbol to assign.
2. Implement the pure symbol and unit clause heuristics.  If <code>heuristic_level=2</code>, then these heuristics should be used to find select first a pure symbol, and if one is not found, then a unit clause.  If neither pure symbols nor unit clauses are found, then the degree heuristic should be used to select a symbol.  If there are multiple pure symbols or unit clauses found, then use alphabetical order to select amongst them.
3. Modify the pure symbol heuristic to choose the pure symbol that occurs in the most number of clauses.  This should be activated if <code>heuristic_level=3</code>.

<h1>Basic Functionality Tests</h1>

All of the tests in this section must be passed for the code to be considered fully functional.  Other unseen test cases will be used after submission to further verify functionality

In [4]:
# This section will test the correct definition of the literal class


# Test the name attribute works correctly
assert(A.name == 'A')

# Test that negation works correctly
assert(not (-C).sign)

In [5]:
# This section will test that the KB has been correctly converted to CNF by performing 
# tests over all possible models
import itertools
symbols = {A,B,C,D,E,F}
symbol_list = [A,B,C,D,E,F]

def evaluate_russell_norvig_KB(model):
    # Manually evaluate the KB using the model
    s = list()
    s.append(model[A] == (model[B] or model[E]))
    s.append(model[E] <= model[D])
    s.append((model[C] and model[F]) <= (not model[B]))
    s.append(model[E] <= model[B])
    s.append(model[B] <= model[F])
    s.append(model[B] <= model[C])
    return all(s)

def evaluate_KB(KB, model):
    model_true = True
    model_name_dict = {l.name: model[l] for l in model}
    for clause in KB:
        evaluation = {l.sign == model_name_dict[l.name] for l in clause if l.name in model_name_dict}
        model_true = model_true and any(evaluation)
    return model_true

all_models = list(itertools.product([False, True], repeat=6))
for i in range(0, len(all_models)):
    # Test all truth values
    test_model = {symbol_list[j]: all_models[i][j] for j in range(0,6)}
    
    # Test the model
    assert(evaluate_russell_norvig_KB(test_model) == evaluate_KB(KB, test_model))

In [6]:
# This section will test the basic working of DPLL
# Satisfiable model
test_KB = [{A, C},{-A, C}, {B, -C}]
t, model, symbol_trace = DPLL_Satisfiable(test_KB)
assert(evaluate_KB(test_KB, model))
assert(t)

# Unsatisfiable model
test_KB = [{A, C},{-A, C}, {-C}]
t, model, symbol_trace = DPLL_Satisfiable(test_KB)
assert(not t)

In [7]:
# This will test DPLL on the KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,0)
print(model)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model)) 

{B: False, A: False, D: False, F: False, C: False, E: False}


<h1>Extension Tests</h1>

This section will test that the degree heuristic, pure symbol and unit clause heuristics are correctly implemented.  Note that in your code, the <code>heuristic_level</code> needs to correctly activate the heuristic being tested, and the <code>symbol_list</code> needs to be correctly generated 

In [8]:
# Degree Heuristic Test for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,1)
print(model)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))  

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] in [['B','F', 'E', 'C', 'C', 'E', 'A'], ['B', 'A', 'C', 'F', 'E', 'A']])

symbol_trace

{B: False, A: False, C: False, F: False, E: False}


[B, A, C, F, E, A]

In [9]:
# Pure Symbol & Unit Clause Heuristic Test for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,2)
print(model)


# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] in [['D', 'B', 'E', 'A', 'C', 'F', 'E', 'A'], ['D', 'B', 'A', 'F', 'C', 'E', 'A'], ['D', 'B', 'A', 'C', 'F', 'E', 'A']])

{D: True, B: False, A: False, C: False, F: False, E: False}


In [10]:
# Pure Symbol & Unit Clause Heuristic Test, choosing the most-frequently used pure symbol, for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,3)
print(model)
print(symbol_trace)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] in [['D', 'B', 'A', 'F', 'C', 'E', 'A'], ['D', 'B', 'A', 'C', 'F', 'E', 'A']])

{D: True, B: False, A: False, C: False, F: False, E: False}
[D, B, A, C, F, E, A]
